In [0]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
import time
import numpy as np

In [0]:
data_train = pd.read_csv('train.csv')

In [3]:
data_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
data_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
#Substituindo os NaN da coluna 'idade' usando o algoritmo KNN.
from sklearn.impute import KNNImputer
fillna_knn = KNNImputer(n_neighbors = 2, weights="uniform",copy=False)
data_train['Age'] = fillna_knn.fit_transform(data_train[['Age']])
data_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
data_train.groupby('Survived')['Sex'].value_counts()
data_train['Sex'].replace(['male','female'],[0,1], inplace = True)
data_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S


In [0]:
x = data_train[['Age','Pclass','Sex','SibSp','Parch']]
y = data_train['Survived']

In [8]:
SEED = 42
np.random.seed(SEED)
params = {
  'n_neighbors' : range(3,30),
  'weights': ['uniform','distance'],
  'algorithm' : ['ball_tree','kd_tree','brute'],
}
tic = time.time()
search =  GridSearchCV(KNeighborsClassifier(),
                   params,
                   cv = KFold(n_splits=10))

search.fit(x,y)
tac = time.time()
delta_t = tac - tic
print('time:%.2f seconds'% delta_t)
results = pd.DataFrame(search.cv_results_)
results_sorted = results.sort_values('mean_test_score', ascending= False)
results_sorted.head()

time:10.96 seconds


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
61,0.002246,0.000069,0.001709,0.000042,kd_tree,6,distance,"{'algorithm': 'kd_tree', 'n_neighbors': 6, 'we...",0.766667,0.842697,0.764045,0.808989,0.831461,0.752809,0.831461,0.797753,0.820225,0.797753,0.801386,0.029857,1
119,0.002689,0.000040,0.003828,0.000116,brute,8,distance,"{'algorithm': 'brute', 'n_neighbors': 8, 'weig...",0.777778,0.808989,0.752809,0.808989,0.820225,0.752809,0.831461,0.808989,0.820225,0.808989,0.799126,0.026592,2
115,0.002700,0.000059,0.003759,0.000103,brute,6,distance,"{'algorithm': 'brute', 'n_neighbors': 6, 'weig...",0.755556,0.842697,0.752809,0.786517,0.820225,0.730337,0.842697,0.808989,0.831461,0.808989,0.798027,0.037953,3
7,0.002175,0.000053,0.001809,0.000057,ball_tree,6,distance,"{'algorithm': 'ball_tree', 'n_neighbors': 6, '...",0.766667,0.842697,0.764045,0.808989,0.820225,0.741573,0.831461,0.797753,0.808989,0.797753,0.798015,0.030285,4
11,0.002189,0.000142,0.001863,0.000039,ball_tree,8,distance,"{'algorithm': 'ball_tree', 'n_neighbors': 8, '...",0.766667,0.831461,0.764045,0.820225,0.842697,0.752809,0.842697,0.786517,0.797753,0.775281,0.798015,0.032302,4


In [0]:
model = search.best_estimator_

# Testando o modelo

In [10]:
data_test = pd.read_csv('test.csv')
data_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [11]:
data_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [12]:
#susbtituindo os NaN da coluna idade
fillna_knn = KNNImputer(n_neighbors=2, weights="uniform",copy=False)
data_test['Age'] = fillna_knn.fit_transform(data_test[['Age']])
data_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [13]:
data_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.50000,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.00000,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.00000,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.00000,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.00000,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,30.27259,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.00000,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.50000,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,30.27259,0,0,359309,8.0500,NaN,S


In [14]:
data_test['Sex'].replace(['male','female'],[0,1], inplace = True)
data_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",0,34.50000,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.00000,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",0,62.00000,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",0,27.00000,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.00000,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",0,30.27259,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",1,39.00000,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",0,38.50000,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",0,30.27259,0,0,359309,8.0500,NaN,S


In [0]:
x_test = data_test[['Age','Pclass','Sex','SibSp','Parch']]

In [0]:
y = model.predict(x_test)

In [17]:
# vendo como deve ser a submissão
example_of_submission = pd.read_csv('gender_submission.csv')
example_of_submission

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [0]:
# criando a tabela para a submissão
my_submission = pd.DataFrame(data_test['PassengerId'])
my_submission['Survived'] = y

In [0]:
my_submission.to_csv('my_submission.csv', index = False)

In [0]:
test1 = pd.read_csv('my_submission.csv')
# O score foi 0.7 segundo o algoritmo do keagle.